In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
from matplotlib import animation
from IPython.display import HTML

from collections import defaultdict
from itertools import zip_longest
from functools import partial

## reference : 
1. http://louistiao.me/notes/visualizing-and-animating-optimization-algorithms-with-matplotlib/
2. http://ruder.io/optimizing-gradient-descent/index.html#gradientdescentoptimizationalgorithms

우리는 두 변수(X,Y)에 대한 손실함수가 아래와 같은 수식이라고 생각해보자

$ Loss(x,y) = (1.5-x+x*y)^2 + (2.25-x+x*y^2)^2+(2.625-x+x*y^3)^2$

(사실 이 수식은 Optimization을 평가하는 테스트 Function 중 하나)

[Test Functions for Optimization](https://en.wikipedia.org/wiki/Test_functions_for_optimization)

다른 수식으로는 
* $Loss(x,y) = [1+(x+y+1)^2(19-14x+3x^2-14y+6xy+3y^2)][30+(2x-3y)^2(18-32x+12x^2+48y-36xy+27y^2)]$

```python
loss = lambda x,y : (
    (1+(x+y+1)**2*(19-14*x+3*x**2-14*y+6*x*y+3*y**2)
     *(30+(2*x-3*y)**2*(18-32*x+12*x**2+48*y-36*x*y+27*y**2)))
)

xmin, xmax, xstep = -2.0, 2.0, 0.1
ymin, ymax, ystep = -2.0, 2.0, 0.1

global_minimum = np.array([[0.0],[-1],[3.0]])
init_point = np.array([[-1.5],[1.5],[800000]])
```

In [ ]:
loss = lambda x,y : ((1.5-x+x*y)**2
                     +(2.25-x+x*y**2)**2
                     +(2.625-x+x*y**3)**2)

xmin, xmax, xstep = -4.5, 4.5, 0.1
ymin, ymax, ystep = -4.5, 4.5, 0.1
global_minimum = np.array([[3],[0.5],[0]])
init_point = np.array([[3.0],[4.0],[39063]])

In [ ]:
# meshgrid는 벡터 x 및 y에 포함된 좌표를 바탕으로 순서대로, 2차원 좌표를 반환
xs, ys = np.meshgrid(np.arange(xmin,xmax+xstep,xstep),
         np.arange(ymin,ymax+ystep,ystep))

zs = loss(xs,ys) # zs 그리기

### 손실함수를 3차원 공간에 맵핑

In [ ]:
fig = plt.figure(figsize=(8, 5))
ax = plt.axes(projection='3d', elev=50, azim=-50)

ax.plot_surface(xs, ys, zs, norm=LogNorm(), rstride=1, cstride=1, 
                edgecolor='none', alpha=.8, cmap=plt.cm.jet)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

ax.set_xlim((xmin, xmax))
ax.set_ylim((ymin, ymax))

plt.show()

우리는 위와 같은 그래프 아래에서 이 그래프에서의 최솟값은 0이고, 

최솟값이 되는 곳의 좌표점(x,y)는 바로 (3.0,0.5)이므로 그걸 먼저 위에 찍어보자

In [ ]:
fig = plt.figure(figsize=(8, 5))
ax = plt.axes(projection='3d', elev=50, azim=-50)

ax.plot_surface(xs, ys, zs, norm=LogNorm(), 
                rstride=1, cstride=1, 
                edgecolor='none', alpha=.8, 
                cmap=plt.cm.jet)

ax.plot(*global_minimum, 'r*', markersize=20)

ax.plot(*init_point, 'g*', markersize=20)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')

ax.set_xlim((xmin, xmax))
ax.set_ylim((ymin, ymax))

plt.show()

### 이걸 윗 방향에서 본다고 해보자

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.contour(xs, ys, zs, levels=np.logspace(0, 5, 35), 
           norm=LogNorm(), cmap=plt.cm.jet)
ax.plot(global_minimum[0],global_minimum[1], 'r*', markersize=18)

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')

ax.set_xlim((xmin, xmax))
ax.set_ylim((ymin, ymax))

plt.show()

마치 등고선 처럼 그림이 그려지게 된다

In [ ]:
def plot_2d_loss_function(xs, ys, zs, path=None):
    """
    3차원 손실 함수를 2차원 등고선 형태로 나타내는 메소드
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.contour(xs, ys, zs, levels=np.logspace(0, 5, 35), 
               norm=LogNorm(), cmap=plt.cm.jet)

    if path is not None:
        ax.quiver(path[0,:-1], path[1,:-1], 
                  path[0,1:]-path[0,:-1], path[1,1:]-path[1,:-1], 
                  scale_units='xy', angles='xy', scale=1, color='k')
        
    ax.plot(global_minimum[0],global_minimum[1],
            'r*', markersize=18)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')

    ax.set_xlim((xmin, xmax))
    ax.set_ylim((ymin, ymax))
    
    return fig, ax

In [ ]:
class TrajectoryAnimation(animation.FuncAnimation):
    def __init__(self, paths, labels=[], fig=None, ax=None, frames=None, 
                 interval=60, repeat_delay=5, blit=True, **kwargs):

        if fig is None:
            if ax is None:
                fig, ax = plt.subplots()
            else:
                fig = ax.get_figure()
        else:
            if ax is None:
                ax = fig.gca()

        self.fig = fig
        self.ax = ax
        
        self.paths = paths

        if frames is None:
            frames = max(path.shape[1] for path in paths)
  
        self.lines = [ax.plot([], [], label=label, lw=2)[0] 
                      for _, label in zip_longest(paths, labels)]
        self.points = [ax.plot([], [], 'o', color=line.get_color())[0] 
                       for line in self.lines]
        self.ax.legend(loc='upper left')
        super(TrajectoryAnimation, self).__init__(fig, self.animate, init_func=self.init_anim,
                                                  frames=frames, interval=interval, blit=blit,
                                                  repeat_delay=repeat_delay, **kwargs)

    def init_anim(self):
        for line, point in zip(self.lines, self.points):
            line.set_data([], [])
            point.set_data([], [])
        return self.lines + self.points

    def animate(self, i):
        for line, point, path in zip(self.lines, self.points, self.paths):
            line.set_data(*path[:2,:i])
            point.set_data(*path[:2,i-1:i])
        return self.lines + self.points

### Optimizer 별로 어떤 식으로 움직이는 지 확인해보자

### Vanilla SGD

In [ ]:
learning_rate = 1e-5
n_epoch = 2000
frame = 50

In [ ]:
graph = tf.Graph()

with graph.as_default():
    x = tf.Variable(init_point[0,0])
    y = tf.Variable(init_point[1,0])

    loss = ((1.5-x+x*y)**2
             +(2.25-x+x*y**2)**2
             +(2.625-x+x*y**3)**2)
    
    train_op = (tf.train.
                GradientDescentOptimizer(learning_rate).
                minimize(loss))

In [ ]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    path_ = []
    for i in range(n_epoch):
        if i % 50 == 0:
            p_x = x.eval(sess)
            p_y = y.eval(sess)
            p_z = sess.run(loss)
            point = [p_x, p_y, p_z] # 현재 x,y를 구함
            path_.append(point)    
            
        sess.run(train_op) # 학습
        
    sgd_path = np.array(path_).T

In [ ]:
fig, ax = plot_2d_loss_function(xs,ys,zs,sgd_path)

#### Learning Rate에 따라 어떻게 달라지는가

In [ ]:
def get_sgd_path(learning_rate):
    graph = tf.Graph()

    with graph.as_default():
        x = tf.Variable(init_point[0,0])
        y = tf.Variable(init_point[1,0])

        loss = (1.5 - x + x*y)**2+(2.25-x+x*y**2)**2+(2.625-x+x*y**3)*2    

        train_op = (tf.train.
                    GradientDescentOptimizer(learning_rate).
                    minimize(loss))

    with tf.Session(graph=graph) as sess:
        sess.run(tf.global_variables_initializer())
        path_ = []
        for i in range(n_epoch):
            if i % 50 == 0:
                p_x = x.eval(sess)
                p_y = y.eval(sess)
                p_z = sess.run(loss)
                point = [p_x, p_y, p_z] # 현재 x,y를 구함
                path_.append(point)    

            sess.run(train_op) # 학습

        sgd_path = np.array(path_).T
    return sgd_path

In [ ]:
path_3 = get_sgd_path(1e-3)
path_4 = get_sgd_path(1e-4)
path_5 = get_sgd_path(1e-5)

In [ ]:
fig, ax = plot_2d_loss_function(xs,ys,zs)
anim = TrajectoryAnimation([path_3,path_4,path_5],
                           ["1e-3","1e-4","1e-5"], fig=fig,ax=ax)

HTML(anim.to_html5_video())

### Momentum Gradient Descent

In [ ]:
learning_rate = 1e-4
momentum = 0.8
n_epoch = 2000
frame = 50

In [ ]:
graph = tf.Graph()

with graph.as_default():
    x = tf.Variable(init_point[0,0])
    y = tf.Variable(init_point[1,0])

    loss = (1.5 - x + x*y)**2+(2.25-x+x*y**2)**2+(2.625-x+x*y**3)*2    
    
    train_op = (tf.train.MomentumOptimizer(learning_rate, momentum).
                minimize(loss))

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    path_ = []
    for i in range(n_epoch):
        if i % 50 == 0:
            p_x = x.eval(sess)
            p_y = y.eval(sess)
            p_z = sess.run(loss)
            point = [p_x, p_y, p_z] # 현재 x,y를 구함
            path_.append(point)    
            
        sess.run(train_op) # 학습
        
    momentum_path = np.array(path_).T

In [ ]:
fig, ax = plot_2d_loss_function(xs,ys,zs,momentum_path)

In [ ]:
def get_momentum_path(learning_rate,momentum):
    graph = tf.Graph()

    with graph.as_default():
        x = tf.Variable(init_point[0,0])
        y = tf.Variable(init_point[1,0])

        loss = (1.5 - x + x*y)**2+(2.25-x+x*y**2)**2+(2.625-x+x*y**3)*2    

        train_op = (tf.train.
                    MomentumOptimizer(learning_rate,momentum).
                    minimize(loss))

    with tf.Session(graph=graph) as sess:
        sess.run(tf.global_variables_initializer())
        path_ = []
        for i in range(n_epoch):
            if i % 50 == 0:
                p_x = x.eval(sess)
                p_y = y.eval(sess)
                p_z = sess.run(loss)
                point = [p_x, p_y, p_z] # 현재 x,y를 구함
                path_.append(point)    

            sess.run(train_op) # 학습

        path = np.array(path_).T
    return path

In [ ]:
path_1 = get_momentum_path(learning_rate,0.1)
path_5 = get_momentum_path(learning_rate,0.5)
path_8 = get_momentum_path(learning_rate,0.8)
path_9 = get_momentum_path(learning_rate,0.9)

In [ ]:
fig, ax = plot_2d_loss_function(xs,ys,zs)
anim = TrajectoryAnimation([path_1,path_5,path_8,path_9],
                           ["0.1","0.5","0.8","0.9"], fig=fig,ax=ax)

HTML(anim.to_html5_video())

### RMSPropOptimizer 보기

In [ ]:
learning_rate = 1e-2
decay = 0.9
n_epoch = 2000
frame = 50

In [ ]:
graph = tf.Graph()

with graph.as_default():
    x = tf.Variable(init_point[0,0])
    y = tf.Variable(init_point[1,0])

    loss = (1.5 - x + x*y)**2+(2.25-x+x*y**2)**2+(2.625-x+x*y**3)*2    
    
    train_op = (tf.train.
                RMSPropOptimizer(learning_rate,decay=decay).
                minimize(loss))

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    path_ = []
    for i in range(n_epoch):
        if i % 50 == 0:
            p_x = x.eval(sess)
            p_y = y.eval(sess)
            p_z = sess.run(loss)
            point = [p_x, p_y, p_z] # 현재 x,y를 구함
            path_.append(point)    
            
        sess.run(train_op) # 학습
        
    rmsp_path = np.array(path_).T

In [ ]:
fig, ax = plot_2d_loss_function(xs,ys,zs,rmsp_path)

In [ ]:
def get_rmsp_path(learning_rate,decay):
    graph = tf.Graph()

    with graph.as_default():
        x = tf.Variable(init_point[0,0])
        y = tf.Variable(init_point[1,0])

        loss = (1.5 - x + x*y)**2+(2.25-x+x*y**2)**2+(2.625-x+x*y**3)*2    

        train_op = (tf.train.
                    RMSPropOptimizer(learning_rate,decay=decay).
                    minimize(loss))

    with tf.Session(graph=graph) as sess:
        sess.run(tf.global_variables_initializer())
        path_ = []
        for i in range(n_epoch):
            if i % 50 == 0:
                p_x = x.eval(sess)
                p_y = y.eval(sess)
                p_z = sess.run(loss)
                point = [p_x, p_y, p_z] # 현재 x,y를 구함
                path_.append(point)    

            sess.run(train_op) # 학습

        rmsp_path = np.array(path_).T
    return rmsp_path

In [ ]:
path_99 = get_rmsp_path(learning_rate,decay=0.99)
path_9  = get_rmsp_path(learning_rate,decay=0.9)
path_5  = get_rmsp_path(learning_rate,decay=0.5)
path_1  = get_rmsp_path(learning_rate,decay=0.1)

In [ ]:
fig, ax = plot_2d_loss_function(xs,ys,zs)
anim = TrajectoryAnimation([path_1,path_5,path_9,path_99],
                           ["0.1","0.5","0.9","0.99"], fig=fig,ax=ax)

HTML(anim.to_html5_video())

### ADAM Optimizer

In [ ]:
learning_rate = 1e-1
n_epoch = 2000
frame = 50

In [ ]:
graph = tf.Graph()

# 첫 위치를 (3.0, 4.0)으로 잡아보자
with graph.as_default():
    x = tf.Variable(init_point[0,0])
    y = tf.Variable(init_point[1,0])

    loss = (1.5 - x + x*y)**2+(2.25-x+x*y**2)**2+(2.625-x+x*y**3)*2    
    
    train_op = (tf.train.AdamOptimizer(learning_rate).
                minimize(loss))

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    path_ = []
    for i in range(n_epoch):
        if i % 50 == 0:
            p_x = x.eval(sess)
            p_y = y.eval(sess)
            p_z = sess.run(loss)
            point = [p_x, p_y, p_z] # 현재 x,y를 구함
            path_.append(point)    
            
        sess.run(train_op) # 학습
        
    adam_path = np.array(path_).T

In [ ]:
fig, ax = plot_2d_loss_function(xs,ys,zs,adam_path)